In [1]:
import os
import json
import rasterio
import numpy as np
import geopandas as gpd

from glob import glob
from rasterio.features import shapes
from shapely.geometry import shape, box
from geopandas import GeoSeries, GeoDataFrame

In [2]:
def filter_raster_with_threshold(raster, threshold, output_file):
    # load merged classified raster -> vectorize
    im = rasterio.open(raster)
    data = im.read(1)

    data[data <= threshold] = 0
    data[data > threshold] = 1

    meta_data = im.profile

    with rasterio.open(output_file, 'w', **meta_data) as dst:
        dst.write(data, 1)

In [ ]:
threshold = 80
raster = '/home/terraloupe/Downloads/leverkusen_showcase/stop_line/segmentation_results_stop_line_2nd_iteration_v1_leverkusen_showcase_nrw_dop10_319999_5622000.tif'
output_file = '/home/terraloupe/Downloads/leverkusen_showcase/stop_line/cutoff_80.tif'
filter_raster_with_threshold(raster, threshold, output_file)

In [ ]:
def sign_category_dataframe(file, output_name, output_dir, threshold):
    
    # read the un-filtered data frame
    df = gpd.read_file(file)
    df_out = []
    
    # filter the None and sign category
    for index, row in df.iterrows():
        if row['geometry'] is None:
            print('remove None')
            continue
        if threshold is not None:
            if row['score'] < threshold:
                continue
        #if row['category'] == 'Lane Marking':
        row_out = {'category_num':1, 'category': 'Road Arrow', 'geometry':row['geometry']}
        df_out.append(row_out)
    
    # apply crs to data frame
    df_out = GeoDataFrame(df_out)
    df_out.crs = {'init': 'epsg:%s' % 25832}
    
    # save the output
    df_out.to_file(os.path.join(output_dir, output_name), driver='GeoJSON')

In [ ]:
file = 'input.geojson'
output_name = 'pred_single_0.5.geojson'
output_dir = 'output_folder/'
threshold = 0.5
sign_category_dataframe(file, output_name, output_dir, threshold)

In [5]:
def normalise_dataframe(input_geo_file, output_name, output_dir, threshold=None):
    df = gpd.read_file(input_geo_file)
    bbox = []
    for _, row in df.iterrows():
        if row.geometry is None:
            continue
        if row.category not in list(mydict.keys()):
            continue
        if not row.geometry.is_valid:
            continue
        if threshold is not None:
            if row['score'] < threshold:
                continue
                
        x1, y1, x2, y2 = row.geometry.bounds
        bbox.append(gpd.GeoSeries({'category':row['category'], 
                                   'geometry': box(x1, y1, x2, y2), 
                                   'category_num':int(mydict[row['category']])}))
    bbox_df = gpd.GeoDataFrame(bbox)
    bbox_df.crs = df.crs
    
    #bbox_df.to_file(os.path.join(output_dir, output_name), driver='GeoJSON')
    bbox_df.to_file(os.path.join(output_dir, output_name), driver='ESRI Shapefile')
    #write_json(bbox_df, output_name, output_dir)

In [6]:
mydict = {'Left Arrow': '5',
          'Left-Right Arrow': '6',
          'Right Arrow': '4',
          'Slightly-Left Arrow': '7',
          'Slightly-Right Arrow': '8',
          'Straight Arrow': '1',
          'Straight-Left Arrow': '2',
          'Straight-Right Arrow': '3',
          'background': '0'}
class_mapping = {0: 'background',
                 1: 'Straight Arrow',
                 2: 'Straight-Left Arrow',
                 3: 'Straight-Right Arrow',
                 4: 'Right Arrow',
                 5: 'Left Arrow',
                 6: 'Left-Right Arrow',
                 7: 'Slightly-Left Arrow',
                 8: 'Slightly-Right Arrow'}

In [7]:
input_geo_file = '/home/terraloupe/Downloads/leverkusen_showcase/road_arrow/compiled_output.geojson'
output_name = 'output_score_0.3.shp'
output_dir = '/home/terraloupe/Downloads/leverkusen_showcase/road_arrow/'
threshold = 0.3
normalise_dataframe(input_geo_file, output_name, output_dir, threshold)